## Modelling the diurnal cycle

COMETS can simulate periodically changing environments, where the periodic function can be either a step function, a sine function or a half sine function. The most obvious use case for this functionality is to study how the metabolism of photosynthetic organisms change during the day / night cycle with varying sunlight (photons) and how this affects the microbes. Here we simulate one such experiment with a genome-scale model of Prochlorococcus, the most abundant marine photoautotroph.

### Import the models and medium conditions, define light and light absorption parameters and initialize the COMETS layout

The first step is to load the Prochlorococcus model iSO595.

In [1]:
import comets as c
import math
import matplotlib.pyplot as plt

# Load Prochlorococcus Genome-scale model
model = c.model('models/iSO595v6.xml');
model.initial_pop = [1, 1, 1e-7]
model.obj_style = 'MAX_OBJECTIVE_MIN_TOTAL'

Using license file /home/djordje/gurobi.lic
Academic license - for non-commercial use only


We now define the parameters related to light absorption and use these to calculate a model-specific absorption coefficient. We assume a monochromatic light source at 680 nm, but it is possible to extend these calculations to a light source with a spectral distribution. All parameters are acquired from the literature(refs 52, 53, 54, 55, 56, 57). 

In [2]:
# The ratio of chlorophyll is extracted from the model biomass-function
ci_dvchla = 0.016                 # gr/gDW (Partensky 1993 / Casey 2016)
ci_dvchlb = 0.0013                # gr/gDW (Partensky 1993 / Casey 2016)
absorption_dvchla_680 = 0.0184    # m^2 mg^-1 (Bricaud et al., 2004)
absorption_dvchlb_680 = 0.0018    # m^2 mg^-1 (Bricaud et al., 2004)
absorption_water_680 = 0.465      # m^-1 (Pope and Fry, 1997)
wavelength = 680                  # nm

Calculate the packaging effect. This is taking into account that the light-absorbing pigments are not dissolved in the media, but contained within discrete cells 58. The packaging effect approaches 0 asymptotically for large cells. Depending on the accuracy needed in the calculation the packaging effect can be assumed  to be close to 1 for very small cells, such as Prochlorococcus 58.

In [3]:
diameter = 0.6                    # um (Morel et al., 1993)
n_dash = 13.77*1e-3               # imaginary part of refractive index at 675 nm (Stramski et al. 2001)
size_parameter_alpha = diameter*1e3*math.pi/wavelength    # The ratio between the cell size and wavelength
rho_dash = 4*size_parameter_alpha*n_dash
Q_a = 1+(2*math.exp(-rho_dash)/rho_dash)+2*(math.exp(-rho_dash)-1)/rho_dash**2
packaging_effect = 1.5*Q_a/rho_dash

# Calculate the Prochlorococcus specific biomass absorption coefficient in units m2/ g DW biomass
absorption_biomass = packaging_effect*(ci_dvchla*1e3*absorption_dvchla_680+ci_dvchlb*1e3*absorption_dvchlb_680)

Set the calculated absorption rate as a model parameter for the exchange reaction of light. `LightEX` is the exchange reaction for photons in the model.

In [4]:
model.add_light('LightEX', absorption_biomass, absorption_water_680)

Now create the layout and define concentration of metabolites in the growth medium. We here set the concentration of all essential metabolites except photons to 1000 mmol and activate the `static` flag, because we want the growth to be only limited by the light conditions

In [5]:
# Make layout with the COMETS toolbox
layout = c.layout(model);

# Define medium
metabs = ['Ammonia[e]', 'HCO3[e]', 'CO2[e]', 'H[e]', 'Orthophosphate[e]', 'H2O[e]', 'Cadmium[e]', 
          'Calcium_cation[e]', 'Chloride_ion[e]', 'Cobalt_ion[e]', 'Copper[e]', 'Fe2[e]', 
          'Magnesium_cation[e]', 'Molybdenum[e]', 'K[e]','Selenate[e]', 'Sodium_cation[e]', 
          'Strontium_cation[e]', 'Sulfate[e]', 'Zn2[e]', 'Hydrogen_sulfide[e]']

for i in metabs:
    layout.set_specific_metabolite(i, 1000)
    layout.set_specific_static(i, 1000)

### Define light conditions. 
Light is modelled as individual photons the value is the number of photons (in mmol) absorbed by the organism at each timepoint. While it is treated like any other metabolite, it is more reasonable to consider it light intensity than a concentration which the other compounds in the media are. To model natural light conditions we use the periodic function called half sin which is equal to $max(f(t), 0) $ where:

$$
f(t) = A\sin(\omega t+\phi)+C
$$

Here A is the amplitude, $\omega$ is the angular frequency ($\omega=2\pi/T$), T the period, $\phi$ the phase and C the offset. Other periodic functions are available: step function, sine and cosine. In this example we define global light conditions. We here define the period to 24 hours with an amplitude of 0.04 mmol photons per meter squared per second.

In [6]:
# Set light conditions by defining parameters
layout.set_global_periodic_media(metabolite='Photon[e]', function='half_sin', amplitude=0.04,
                                 period=24, phase=0, offset=0)

### Set simulation parameters

Now set relevant simulation parameters

In [7]:
# Set simulation parameters
sim_params = c.params()

sim_params.all_params['maxCycles'] = 480
sim_params.all_params['timeStep'] = 0.1
sim_params.all_params['defaultDiffConst'] = 0

### Run `COMETS` simulation

In [10]:
# Runs comets and produce the output files mediaLog.m and biomassLog.m
simulation = c.comets(layout, sim_params)
simulation.JAVA_CLASSPATH = '/home/djordje/Dropbox/COMETS_RUN/lib/jmatio.jar:/home/djordje/Dropbox/COMETS_RUN/lib/jdistlib-0.4.5-bin.jar:/home/djordje/Dropbox/COMETS_RUN/lib/commons-math3-3.6.1.jar:/home/djordje/Dropbox/COMETS_RUN/lib/commons-lang3-3.9.jar:/home/djordje/Dropbox/COMETS_RUN/lib/colt.jar:/home/djordje/Dropbox/COMETS_RUN/lib/concurrent.jar:/home/djordje/Dropbox/COMETS_RUN/bin/comets_2.8.2.jar:/opt/gurobi901/linux64/lib/gurobi.jar'
simulation.run(delete_files=False)

These are the expected locations for dependencies:
Dependency 			 expected path
__________ 			 _____________
junit			/home/djordje/Dropbox/COMETS_RUN/lib/junit/junit-4.12.jar
hamcrest			/home/djordje/Dropbox/COMETS_RUN/lib/junit/hamcrest-core-1.3.jar
jogl_all			/home/djordje/Dropbox/COMETS_RUN/lib/jogl/jogamp-all-platforms/jar/jogl-all.jar
gluegen_rt			/home/djordje/Dropbox/COMETS_RUN/lib/jogl/jogamp-all-platforms/jar/gluegen-rt.jar
gluegen			/home/djordje/Dropbox/COMETS_RUN/lib/jogl/jogamp-all-platforms/jar/gluegen.jar
gluegen_rt_natives			/home/djordje/Dropbox/COMETS_RUN/lib/jogl/jogamp-all-platforms/jar/gluegen-rt-natives-linux-amd64.jar
jogl_all_natives			/home/djordje/Dropbox/COMETS_RUN/lib/jogl/jogamp-all-platforms/jar/jogl-all-natives-linux-amd64.jar
jmatio			/home/djordje/Dropbox/COMETS_RUN/lib/JMatIO/lib/jamtio.jar
jmat			/home/djordje/Dropbox/COMETS_RUN/lib/JMatIO/JMatIO-041212/lib/jmatio.jar
concurrent			/home/djordje/Dropbox/COMETS_RUN/lib/colt/lib/concurrent.jar
colt			/ho

FileNotFoundError: [Errno 2] No such file or directory: 'total_biomass_log_0x7fc692895f28'

In [11]:
print(simulation.run_output)

-script
running script file: /home/djordje/Dropbox/projects/COMETS-Python-Toolbox/.current_script
Loading layout file '/home/djordje/Dropbox/projects/COMETS-Python-Toolbox/.current_layout'...
Found 1 model files!
Loading '/home/djordje/Dropbox/projects/COMETS-Python-Toolbox/COBRAModel.cmd' ...
Loading '/home/djordje/Dropbox/projects/COMETS-Python-Toolbox/COBRAModel.cmd' ...
Error in model file '/home/djordje/Dropbox/projects/COMETS-Python-Toolbox/COBRAModel.cmd': edu.bu.segrelab.comets.exception.ModelFileException: The LIGHT parameter at line 6182 should be followed by its surface to weight ratio in m^2 per gDW
Num media101
76 half_sin 0.04 24 0 0
Parameters: 
0.04
24.0
0.0
0.0
Constructing world...
Exception in thread "main" java.lang.NullPointerException
	at edu.bu.segrelab.comets.fba.FBAWorld.changeModelsInWorld(FBAWorld.java:919)
	at edu.bu.segrelab.comets.fba.FBAWorld.<init>(FBAWorld.java:211)
	at edu.bu.segrelab.comets.fba.FBACometsLoader.loadLayoutFile(FBACometsLoader.java:657)


### Plot results

Load the results using the biomass and media output dataframes